# Importamos dependencias y variables de entorno

In [7]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

# Load environment variab
from dotenv import load_dotenv
load_dotenv()

import os
import yaml
from crewai import Agent, Task, Crew

OPENAI_API_KEY = os.environ['OPENAI_API_KEY']
SERPER_API_KEY = os.environ['SERPER_API_KEY']
#ANTHROPIC_API_KEY= os.environ['ANTHROPIC_API_KEY']
GROQ_API_KEY = os.environ["GROQ_API_KEY"]

Cargamos los archivos yaml con las instrucciones de agentes y tareas

In [8]:
# Define file paths for YAML configurations
files = {
    'agents': 'config/agents.yaml',
    'tasks': 'config/tasks.yaml'
}

# Load configurations from YAML files
configs = {}
for config_type, file_path in files.items():
    with open(file_path, 'r') as file:
        configs[config_type] = yaml.safe_load(file)

# Assign loaded configurations to specific variables
agents_config = configs['agents']
tasks_config = configs['tasks']

# Herramientas:

Crewai nos proporciona una serie de herramientas muy pontentes para nuestros sistemas de agentes:

SerperDevTool: Utiliza la API de https://serper.dev/ para generar busquedas potenciadas con inteligencia artificial, devolviendo una serie de URLs

ScrapeWebsiteTool: Una herramienta que nos permite scrapear la informacion de una web dada su URL

WebsiteSearchTool: Super potente, nos permite scrapear una web dada su url, cargar una base de datos vectorial en memoria y hacer busquedas RAG a esa base de datos, lo que resultara en informacion mucho mas especifica sobre lo que estamos buscando.

In [9]:
from crewai_tools import SerperDevTool, ScrapeWebsiteTool, WebsiteSearchTool
#from tools.info_activos import info_activo

Establecemos el numero de URLs a 3, para que solo nos devuelva 3 URLs, ya que si no, devolveria 10, que es el valor por defecto. Hay que tener en cuenta que la API de seper tiene una version gratuita de 2500 peticiones, da de sobra para hacer muchas pruebas pero tampoco hay que pasarse.

In [10]:
SerperTool = SerperDevTool(
    n_results=3,
)

# Creando Agentes, tareas y Crew

Aqui vamos a crear los agentes y las tareas, en este caso, habra varias tareas que se podran hacer a la vez, asi que pondremos async_execution=True, lo cual disminuira el tiempo de respuesta. 

La ultima tarea, sera en la que generaremos el reporte basandonos en la informacion que hemos recogido anteriormente, para ello debemos escribir:

context=[recopilacion_informacion_activo, analisis_tecnico_activo, analisis_fundamental_proyectos_empresas]

Esto hara que el agente espere a tener la respuesta del resto de agentes para generar la suya.

Si quereis podeis descomentar el agente para crear graficos para ver como funciona, pero debereis activar docker.

In [11]:
## AGENTES
agente_informacion_activo = Agent (
    config=agents_config['agente_informacion_activo'],
    tools=[SerperTool, WebsiteSearchTool()],
    llm= 'gpt-4o-mini',
)

agente_analisis_tecnico = Agent (
    config=agents_config['agente_analisis_tecnico'],
    tools=[SerperTool, WebsiteSearchTool()],
    llm= 'gpt-4o-mini',
)

agente_analisis_fundamental = Agent (
    config=agents_config['agente_analisis_fundamental'],
    tools=[SerperTool, WebsiteSearchTool()],
    llm= 'gpt-4o-mini',
)


# agente_creacion_graficos = Agent (
#     config=agents_config['agente_creacion_graficos'],
#     allow_code_execution=True,
#     tools=[info_activo],
#     llm= 'gpt-4o-mini',
# )

agente_compilacion_reporte_final = Agent (
    config=agents_config['agente_compilacion_reporte_final'],
    llm= 'gpt-4o-mini',
)

## TAREAS

recopilacion_informacion_activo = Task(
    config=tasks_config['recopilacion_informacion_activo'],
    agent=agente_informacion_activo,
    async_execution=True,
)

analisis_tecnico_activo = Task(
    config=tasks_config['analisis_tecnico_activo'],
    agent=agente_analisis_tecnico,
    async_execution=True,
)

analisis_fundamental_proyectos_empresas = Task(
    config=tasks_config['analisis_fundamental_proyectos_empresas'],
    agent=agente_analisis_fundamental,
    async_execution=True,
)


# creacion_graficos_activo = Task(
#     config=tasks_config['creacion_graficos_activo'],
#     agent=agente_creacion_graficos,
# )

compilacion_reporte_final = Task(
    config=tasks_config['compilacion_reporte_final'],
    agent=agente_compilacion_reporte_final,
    context=[recopilacion_informacion_activo, analisis_tecnico_activo, analisis_fundamental_proyectos_empresas]
    # context=[recopilacion_informacion_activo, analisis_tecnico_activo, analisis_fundamental_proyectos_empresas, creacion_graficos_activo]
)

# Creating Crew
crypto_report_crew = Crew(
  agents=[
    agente_informacion_activo,
    agente_analisis_tecnico,
    agente_analisis_fundamental,
    #agente_creacion_graficos,
    agente_compilacion_reporte_final,
  ],
  tasks=[
    recopilacion_informacion_activo,
    analisis_tecnico_activo,
    analisis_fundamental_proyectos_empresas,
    #creacion_graficos_activo,
    compilacion_reporte_final,
  ],
  verbose=True
)


2024-11-19 23:58:28,961 - 140186504904832 - __init__.py-__init__:538 - WARNING: Overriding of current TracerProvider is not allowed


## Llamamos a la Crew, pasando como input el activo que queremos analizar

Para referenciar el input, si nos fijamos en los archivos .yaml, estamos simplemente rodeando con corchetes la clave del diccionario que pasamos como input, en este caso {activo}

In [12]:
result = crypto_report_crew.kickoff(inputs={
    'activo': 'XRP'
})

In [13]:
from IPython.display import display, Markdown
display(Markdown(result.raw))

# Informe completo sobre XRP (Ripple)

## 1. Introducción
Ripple (XRP) es un criptoactivo creado por Ripple Labs que se enfoca en facilitar las transferencias de dinero entre distintas monedas. Su propuesta de valor radica en ofrecer soluciones de pago rápidas, eficientes y de bajo costo, buscando eliminar las ineficiencias heredadas en las transferencias internacionales de dinero. A diferencia de muchas criptomonedas que utilizan minería, XRP funciona con un libro mayor distribuido que permite a los participantes confirmar las transacciones de manera ágil.

## 2. Análisis del Activo
- **Nombre Completo**: Ripple (XRP)
- **Ticker**: XRP
- **Capitalización de Mercado**: $61,892,650,644 USD
- **Volumen de Trading Diario**: $5,850,125,500 USD
- **Valoración de la Liquidez**: 8/10
- **Rendimiento Histórico**: XRP alcanzó su máximo histórico de $3.84 en enero de 2018 y actualmente se encuentra alrededor de $1.09, lo que refleja una disminución del 71.74% respecto a su máximo histórico.

## 3. Análisis Técnico
El análisis técnico indica que XRP se encuentra en una tendencia alcista, apoyada por niveles clave y medias móviles. A pesar de esta tendencia, se recomienda a los traders que permanezcan alertas ante la posibilidad de una corrección a corto plazo.

## 4. Análisis Fundamental
### Resumen del Proyecto o Empresa
XRP, impulsado por Ripple Labs, está diseñado para mejorar las transferencias de dinero entre diferentes divisas, con un enfoque en la rapidez y la reducción de costos.

### Información Legal
Ripple Labs enfrenta un litigio con la SEC de EE. UU., que argumenta que XRP debe ser considerado como un valor no registrado. Este proceso legal ha creado incertidumbre en el mercado y podría influir significativamente en la viabilidad y adopción de XRP.

### Información Financiera 
Desde su lanzamiento, XRP ha mostrado gran volatilidad, alcanzando niveles de capitalización de mercado significativos. Sus perspectivas futuras están estrechamente relacionadas con el desenlace del litigio y las tendencias en el mercado de criptomonedas. Aunque ha sido adoptado por varias instituciones financieras, enfrenta retos de competencia por parte de otras soluciones emergentes.

### Información Técnica
XRP opera con un protocolo de consenso que permite tiempos de confirmación de transacciones de aproximadamente 3 a 5 segundos, superando el tiempo de confirmación de Bitcoin. Además, sus costos de transacción son bajos, lo que resulta atractivo para las instituciones financieras que buscan eficiencia. Su diseño también proporciona escalabilidad en comparación con otras tecnologías blockchain.

### Alertas y Riesgos
1. **Litigio con la SEC**: La posible resolución del litigio puede impactar drásticamente el futuro de XRP.
2. **Volatilidad del Mercado**: Las fluctuaciones de precio pueden afectar la confianza de los inversores.
3. **Competencia**: Nuevos actores en el mercado de pagos digitales podrían reducir la participación de XRP.
4. **Regulación**: Cambios en la regulación en distintas jurisdicciones podrían influir en su uso.

## 5. Conclusión
XRP se posiciona como una solución innovadora para la transferencia de capital, destacándose por su rapidez y costo-efectividad. Sin embargo, enfrenta desafíos críticos, como litigios legales, alta volatilidad, y competencia creciente. La resolución del conflicto con la SEC es un factor fundamental que determinara su futuro en el mercado de criptomonedas. Es esencial que inversores y la comunidad en general permanezcan atentos a estos acontecimientos mientras consideran los beneficios potenciales de XRP.

## 6. Fuentes
1. [CryptoEQ](https://www.cryptoeq.io/corereports/xrp-abridged) - Accedido el [Fecha]
2. [Investopedia](https://www.investopedia.com/news/what-role-xrp-ripples-products/) - Accedido el [Fecha]
3. [Forbes](https://www.forbes.com/advisor/in/investing/cryptocurrency/xrp-price-prediction/) - Accedido el [Fecha]

Este informe proporciona una visión integral de XRP, combinando análisis técnicos y fundamentales, junto con consideraciones sobre el contexto legal y financiero del criptoactivo.

## Guardamos el contenido en un archivo 

In [14]:
file_name = "output.txt"

# Abre el archivo en modo de escritura y guarda el contenido
with open(file_name, "w") as file:
    file.write(result.raw)

print(f"El contenido se ha guardado correctamente en {file_name}")